In [67]:
import os
import pandas as pd
import numpy as np
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
from scipy import sparse
import util
import matplotlib.pyplot as plt

In [68]:
TRAIN_DIR = "../data/train"

In [69]:
call_set = set([])

In [70]:
def add_to_set(tree):
    for el in tree.iter():
        call = el.tag
        call_set.add(call)

In [71]:
#creating a set of features counting the number of tags
def call_feats(tree, good_calls):
    #Inputs
    #tree - tree object for every file
    #good_calls - list of tags for which we create the features
    call_counter = {}
    for el in tree.iter():
        call = el.tag
        if call not in call_counter:
            call_counter[call] = 0
        else:
            call_counter[call] += 1

    call_feat_array = np.zeros(len(good_calls))
    for i in range(len(good_calls)):
        call = good_calls[i]
        call_feat_array[i] = 0
        if call in call_counter:
            call_feat_array[i] = call_counter[call]
    return call_feat_array

In [72]:
###Creating function for loading data
def create_matrix(start_index, end_index, tags, direc="../data/train"):
    X = None
    classes = []
    ids = []
    i = -1
    for datafile in os.listdir(direc):
        if datafile == '.DS_Store':
            continue
            
        i += 1
        if i < start_index:
            continue
        if i >= end_index:
            break
        id_str, clas = datafile.split('.')[:2]
        ids.append(id_str)
        #adding target class to training data
        try:
            classes.append(util.malware_classes.index(clas))
        except ValueError:
            assert clas == "X"
            classes.append(-1)
            
        #parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
        add_to_set(tree)
        this_row = call_feats(tree, tags)
        if X is None:
            X = this_row 
        else:
            X = np.vstack((X, this_row))
            
    return X, np.array(classes), ids    

#### Data Exploration

In [73]:
#Data exploration
#Loading the files
fileList = os.listdir(TRAIN_DIR)
numFiles = len(fileList)

In [74]:
#List of unique tags
tags = []
for idx in range(numFiles):
    tree = ET.parse(os.path.join(TRAIN_DIR,fileList[idx]))
    for el in tree.iter():
        call = el.tag
        tags.append(call)
    tags = list(np.unique(tags))
unique_tags = np.unique(tags)

In [75]:
#Converting all tags to 'str' from  numpy.string_
unique_tags = [str(tag) for tag in unique_tags]

In [107]:
X_train, t_train, train_ids = create_matrix(0, numFiles,\
                                            unique_tags, TRAIN_DIR)

In [92]:
features_df = pd.DataFrame(X_train,columns=unique_tags)
features_df['class'] = t_train
features_df['id'] = train_ids
#Saving the features dataframe as a new file
features_df.to_csv('../outputs/features_v1.csv')

#### Cleaning and transforming the test data

In [124]:
TEST_DIR = "../data/test"
testFileList = os.listdir(TEST_DIR)
numTestFiles = len(testFileList)
X_test, t_test, test_ids = create_matrix(0, numTestFiles,\
                                            unique_tags, TEST_DIR)

In [125]:
#Ignoring t_train since there is no response variable 
features_test_df = pd.DataFrame(X_test,columns=unique_tags)
features_test_df['class'] = test_ids

In [126]:
features_test_df.head()

,accept_socket,add_netjob,all_section,bind_socket,change_service_config,check_for_debugger,com_create_instance,com_createole_object,com_get_class_object,connect,...,thread,trimmed_bytes,unload_driver,vm_allocate,vm_mapviewofsection,vm_protect,vm_read,vm_write,write_value,class
0,0,0,4,0,0,1,1,0,0,0,...,4,0,0,0,0,35,0,0,0,0015c8c9ff02fea9d0f45692b9eebfb4abff4e42f
1,0,0,5,1,0,1,0,0,0,0,...,5,0,0,7,0,254,0,4,0,001f298a534ae4b0db7f2707169250aa215c3b5f2
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,001f5fdaaa8bbe20303527198d09a30bb7ca3eb50
3,0,0,5,4,0,1,0,0,0,0,...,5,12,0,0,0,71,0,0,0,002ca2c41b649f85c05ae30013436781a932fecc6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,003e109543b4ea22d2bcc1ec309bf2fd34e9a1a1d


In [138]:
features_test_df.to_csv('../outputs/features_test_v1.csv')

#### Random forest classifiers

In [16]:
from sklearn.cross_validation import train_test_split

In [17]:
##loading the training data
#Reading in the training dataset
train = pd.read_csv('../outputs/features_v1.csv')
y = train['class']
train_ids = train['id']
X = train[train.columns[1:-2]]

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, \
                                                    test_size=0.10, random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
RF = RandomForestClassifier(n_estimators=10,min_samples_leaf=100)

In [65]:
RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=100, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [66]:
print genScore(RF,X_valid, y_valid)

0.779935275081


In [22]:
def genScore(RF, X_valid, y_valid):
    #generating the predictions
    pred = RF.predict(X_valid)
    return np.sum(pred == y_valid)/float(len(y_valid))

In [7]:
#Loading the test data
test = pd.read_csv('../outputs/features_test_v1.csv')
test_ids = test['Id']
X_test = test[test.columns[1:-1]]

In [8]:
pred = RF.predict(X_test)

In [9]:
out_df = pd.DataFrame(test_ids, columns=['Id'])
out_df['Prediction'] = pred
out_df = out_df.set_index('Id')

In [137]:
out_df.to_csv('../outputs/RF_prediction.csv')

In [11]:
###Calculating feature importances and visualizing
importances = RF.feature_importances_
num_features = 10
x_val = importances[np.argsort(importances)[::-1][:num_features]]
x_ticks = X_train.columns[np.argsort(importances)[::-1][:num_features]]

In [13]:
fig, ax = plt.subplots(figsize=(10,10))
ax = pd.Series(x_val).plot(kind='bar')
ax.set_xticklabels(x_ticks)
ax.set_title('Feature Importances')
ax.set_xlabel('Features')
ax.set_ylabel('Importances')
plt.savefig('FeatureImportances.png', bbox_inches='tight')

In [76]:
len(unique_tags)

106

In [77]:
unique_tags

['accept_socket',
 'add_netjob',
 'all_section',
 'bind_socket',
 'change_service_config',
 'check_for_debugger',
 'com_create_instance',
 'com_createole_object',
 'com_get_class_object',
 'connect',
 'connect_socket',
 'control_service',
 'copy_file',
 'create_directory',
 'create_file',
 'create_interface',
 'create_key',
 'create_mailslot',
 'create_mutex',
 'create_namedpipe',
 'create_open_file',
 'create_process',
 'create_process_as_user',
 'create_process_nt',
 'create_service',
 'create_socket',
 'create_thread',
 'create_thread_remote',
 'create_window',
 'delete_file',
 'delete_key',
 'delete_service',
 'delete_share',
 'delete_value',
 'destroy_window',
 'download_file',
 'download_file_to_cache',
 'dump_line',
 'enum_handles',
 'enum_items',
 'enum_keys',
 'enum_modules',
 'enum_processes',
 'enum_services',
 'enum_share',
 'enum_subtypes',
 'enum_types',
 'enum_user',
 'enum_values',
 'enum_window',
 'exit_windows',
 'find_file',
 'find_window',
 'get_computer_name',
 'ge